# Data Processing U850/V850 all files
Teagan King, John Truesdale, Katie Dagon
Updated Feb 2022

## Import libraries

In [7]:
import glob
import xarray as xr
import cftime
import geocat.comp as gc
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

### Set up Dask

In [3]:
# Import dask
import dask

# Use dask jobqueue
from dask_jobqueue import PBSCluster

# Import a client
from dask.distributed import Client

# Setup your PBSCluster
nmem='25GB' # specify memory here so it duplicates below
cluster = PBSCluster(
    cores=1, # The number of cores you want
    memory=nmem, # Amount of memory
    processes=1, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='/glade/scratch/$USER/local_dask', # Use your local directory
    resource_spec='select=1:ncpus=1:mem='+nmem, # Specify resources
    account='P93300313', # Input your project ID here, previously this was known as 'project', now is 'account'
    walltime='04:00:00', # Amount of wall time
    interface='ib0', # Interface to use
)

# Scale up
cluster.scale(40)

# Change your url to the dask dashboard so you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

/glade/u/home/tking/.conda/envs/geocat_comp_env/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39766 instead
  warnings.warn(


In [4]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/tking/proxy/39766/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/tking/proxy/39766/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.46:43038,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/tking/proxy/39766/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Read in files and set pressure level(s)

In [8]:
# location of data
datadir = '/glade/scratch/tking/cgnet/rcp85_2086_2100/'

ufile_format = '*.U.*.nc' # U files are b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4, 3hourly avg from 2080-2100
vfile_format = '*.V.*.nc' # V files are b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4, 3hourly avg from 2080-2100
psfile_format = '*.PS.*.nc' # PS files are b.e13.BRCP85C5CN.ne120_g16.003a.cam.h3, 3hourly avg from 2080-2100

# desired pressure level:
plevel = 850.0  # hPa

# use glob.glob() to get actual files, and sort in order to retain correctly matching files!
ufilenames = sorted(glob.glob(datadir+ufile_format))
vfilenames = sorted(glob.glob(datadir+vfile_format))
psfilenames = sorted(glob.glob(datadir+psfile_format))

# make new .nc files but replace U/V with U850/V850
unew_files = []
for filename in ufilenames:
    unew_files.append((filename.replace('.U.', '.U850.')).split('/')[-1])

vnew_files = []
for filename in vfilenames:
    vnew_files.append((filename.replace('.V.', '.V850.')).split('/')[-1])

In [13]:
# ufilenames = ufilenames[6:]
# vfilenames = vfilenames[6:]
# psfilenames = psfilenames[6:]
# unew_files =unew_files[6:]
# vnew_files =vnew_files[6:]

In [14]:
print(ufilenames)
print(vfilenames)
print(psfilenames)
print(vnew_files)
print(unew_files)

['/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U.2086010100Z-2086123121Z.nc', '/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U.2087010100Z-2087123121Z.nc', '/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U.2088010100Z-2088123121Z.nc', '/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U.2089010100Z-2089123121Z.nc', '/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U.2090010100Z-2090123121Z.nc', '/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U.2091010100Z-2091123121Z.nc', '/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U.2092010100Z-2092123121Z.nc', '/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U.2093010100Z-2093123121Z.nc', '/glade/scratch/tking/cgnet/rcp85_2086_2100/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U.20

### Run all files

In [ ]:
# define dummy values for t_bot and phi_sfc
# reasoning for this described in GitHub issue: https://github.com/NCAR/geocat-comp/issues/26
# Hopefully the need for this will be removed soon.
t_bot=xr.DataArray([])
phi_sfc=xr.DataArray([])

for wind in ['U', 'V']:
    for file_index in range(len(psfilenames)):        
        if wind=='U':
            windfile = xr.open_dataset(ufilenames[file_index], chunks={"time": 100, "ncol":10000}) # adjusting chunks for ~100mb size chunks
        elif wind=='V':
            windfile = xr.open_dataset(vfilenames[file_index], chunks={"time": 100, "ncol":10000}) # adjusting chunks for ~100mb size chunks
        psfile = xr.open_dataset(psfilenames[file_index], chunks={"time": 100, "ncol":10000}) # using the same chunk size for consistency

        file_wind = windfile[wind] # wind at each level
        file_PS = psfile['PS'] # surface pressure

        pref = psfile['P0'] # reference pressure

        # hybrid level coordinates
        hyam = windfile['hyam']
        hybm = windfile['hybm']
        
        # interpolate to get correct grid levels using GeoCAT's interpolate hybrid to pressure function:
        plevdata = gc.interpolation.interp_hybrid_to_pressure(file_wind,  # 3d field U/V (time x lev x ncol)
                            file_PS,  # surface pressure (time x ncol)
                            hyam, hybm,  # coefficients to calculate pressure at each level
                            p0=pref.values,  # reference pressure
                            new_levels=np.array([85000], dtype='float32'),  # interpolate to 850 pressure level
                            lev_dim=None,  # lev is default
                            method='log', # use log because pressure falls off logarithmically

                            extrapolate=True, # extrapolate below ground values
                            variable='other',
                            t_bot=t_bot,  # xarray.DataArray Temperature in Kelvin at the lowest layer of the model.
                                             # Not necessarily the same as surface temperature.
                                             # Required if ``extrapolate`` is True.
                            phi_sfc=phi_sfc  # Geopotential in J/kg at the lowest layer of the model.
                                             # Not necessarily the same as surface geopotential.
                                             # Required if ``extrapolate`` is True.
                            )   #TODO: implement dask here

        if wind == 'U':
            file_save = unew_files[file_index]
        elif wind == 'V':
            file_save = vnew_files[file_index]
        plevdata.to_netcdf('/glade/scratch/tking/cgnet/rcp85_2086_2100/{}'.format(file_save))
        print('generated {}'.format(file_save))

/glade/u/home/tking/.conda/envs/geocat_comp_env/lib/python3.10/site-packages/metpy/interpolate/one_dimension.py:142: UserWarning: Interpolation point out of data bounds encountered
  warnings.warn('Interpolation point out of data bounds encountered')


In [6]:
# %%time
# %%bash
# module load nco/4.7.9
# ncremap -m /glade/campaign/cgd/amp/jet/ClimateNet/data_processing/maps/map_ne120_to_0.23x0.31_bilinear.nc -i /glade/scratch/tking/cgnet/rcp85_2086_2100/U850_2080.nc -o /glade/scratch/tking/cgnet/rcp85_2086_2100/U850_2080_regrid.nc

In [ ]:
# %%bash
# module load nco/4.7.9

# RCP 8.5
U850_V850_to_regrid=['',
                     '',
                     '',
                     '']

for file in U850_V850_to_regrid:
    print('ncremap -m /glade/campaign/cgd/amp/jet/ClimateNet/data_processing/maps/map_ne120_to_0.23x0.31_bilinear.nc -i {} -o /glade/scratch/tking/cgnet/rcp85_2086_2100/regridded_U850_V850/{}'.format(file, file))

# paste commands below into terminal... jobs will be killed if done all at once

## Mask invalid values in the array

In [13]:
import numpy.ma as ma

# gridfill operates on a masked array: https://numpy.org/doc/stable/reference/maskedarray.generic.html#module-numpy.ma

### use dask here (dask.array.ma: https://docs.dask.org/en/stable/generated/dask.array.ma.masked_array.html)
plevdata_ma_invalid = ma.masked_invalid(plevdata[:, :, :])
plevdata_ma_invalid.mask

### USE XARRAY MASKING to keep labeled dims .where()


# plevdata_ma_invalid = dask.array.ma.masked_invalid(plevdata[:, :, :])
# type(plevdata_ma_invalid)

array([[[False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False]]])

In [14]:
np.where(plevdata_ma_invalid.mask==True)

(array([0, 0, 0, ..., 2, 2, 2]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([ 15297,  15304,  15305, ..., 763934, 763935, 765005]))

## Test with a Subset of John's test file:

In [90]:
# there are both true and false values in John's test ufile
# Why is this different from subset of actual data, eg actual data doesn't seem to have True's???
print(plevdata_ma_invalid.mask[0,0,15297])
print(plevdata_ma_invalid.mask[0,0,0])

True
False


In [41]:
plevdata_subset_tnf=plevdata_ma_invalid[0,0,15295:15300]

print(plevdata_subset_tnf)

# ubot is different dimensions than plevdata_ma_invalid b/c only has zeroeth level...
ubot_subset_tnf=ubot[0,15295:15300]
print(ubot_subset_tnf.values)

[-4.09454230957591 -3.2498493224813534 -- -4.144254232261338
 -4.551732974980913]
[-27.909664 -27.89686  -27.886927 -27.615683 -27.605026]


In [50]:
ubot_masked = ma.masked_where(~plevdata_subset_tnf.mask, ubot[0,15295:15300])
# plevdata_subset_tnf.mask

[-- -- -27.886926651000977 -- --]


In [57]:
# plv_subset_masked_w_ubot = ubot_masked + plevdata_subset_tnf
# print(plv_subset_masked_w_ubot)

print(plevdata_subset_tnf)
print(ubot_masked)

merged = np.ma.where(plevdata_subset_tnf.mask, ubot_masked, plevdata_subset_tnf)
print(merged)

[-4.09454230957591 -3.2498493224813534 -- -4.144254232261338
 -4.551732974980913]
[-- -- -27.886926651000977 -- --]
[ -4.09454231  -3.24984932 -27.88692665  -4.14425423  -4.55173297]


## Test with John's test file:

In [15]:
ubot=file_1_U[:,29,:]

In [16]:
# print("file_1_U", file_1_U.shape)
# in going to plevdata, the shape changes such that only one level is present because it's at U850...
# print("plevdata is 3 time slices, 1 level, 777602 columns", plevdata.shape)
# print("plevdata_ma_invalid", plevdata_ma_invalid.shape)
# print("ubot is 3 time slices, 777602 columns", ubot.shape)
# print("merged_full is 3 time slices, 777602 columns", merged_full.shape)

ubot_masked_full = ma.masked_where(~plevdata_ma_invalid[:,0,:].mask, ubot)
merged_full = np.ma.where(plevdata_ma_invalid[0].mask, ubot_masked_full, plevdata_ma_invalid[0])
# print(merged_full)

# all_levels_merged_full=np.asarray([])
# for level in range(0,3):
#     ubot_masked_full = ma.masked_where(~plevdata_ma_invalid[:,level,:].mask, ubot)
#     merged_full = np.ma.where(plevdata_ma_invalid[:,level,:].mask, ubot_masked_full, plevdata_ma_invalid[:,level,:])
#     # print(merged_full)
#     all_levels_merged_full.append(merged_full)

In [17]:
# np.where(merged_full.mask==True)
# merged_full should always have a value, but remaining mask could help us mark what was ubot vs orig val

### trying to find where mask is true on original files...

In [20]:
## THIS CELL IS TAKING HOURS TO RUN... all masks are empty...
# plevdata: 'U'  time: 3plev: 1ncol: 777602

import numpy.ma as ma

# plevdata_ma_invalid = ma.masked_invalid(plevdata[0, 0, :]) # change to :,0,: to just use level 0 and all times
# plevdata_ma_invalid.mask

# plevdata_ma_invalid_subset = plevdata_ma_invalid[0,0,:]
# plevdata_ma_invalid_subset.mask

# plevdata_ma_invalid_first_half = ma.masked_invalid(plevdata[0, 0, 3000:17602]) # change to :,0,: to just use level 0 and all times
# # plevdata_ma_invalid_first_half.mask
# np.where(plevdata_ma_invalid_first_half==True)

# # could we instead look for lat/lon of fill values and check lat/lon of plevdata?
# # but it needs to be regridded...

# plevdata_ma_invalid_2_half = ma.masked_invalid(plevdata[0, 0, 17602:90000]) # change to :,0,: to just use level 0 and all times
# # plevdata_ma_invalid_2_half.mask
# np.where(plevdata_ma_invalid_2_half==True)
# # plevdata_ma_invalid.data.shape
# # plevdata.shape

# plevdata_ma_invalid_3_half = ma.masked_invalid(plevdata[0, 0, 90000:300000]) # change to :,0,: to just use level 0 and all times
# # plevdata_ma_invalid_2_half.mask
# np.where(plevdata_ma_invalid_3_half==True)
# # plevdata_ma_invalid.data.shape
# # plevdata.shape

# plevdata_ma_invalid_4_half = ma.masked_invalid(plevdata[0, 0, 300000:500000]) # change to :,0,: to just use level 0 and all times
# # plevdata_ma_invalid_2_half.mask
# np.where(plevdata_ma_invalid_4_half==True)
# # plevdata_ma_invalid.data.shape
# # plevdata.shape

# plevdata_ma_invalid_5_half = ma.masked_invalid(plevdata[0, 0, 500000:700000]) # change to :,0,: to just use level 0 and all times
# # plevdata_ma_invalid_2_half.mask
# np.where(plevdata_ma_invalid_5_half==True)
# # plevdata_ma_invalid.data.shape
# # plevdata.shape

# plevdata_ma_invalid_6_half = ma.masked_invalid(plevdata[0, 0, 700000:777601]) # change to :,0,: to just use level 0 and all times
# # plevdata_ma_invalid_2_half.mask
# np.where(plevdata_ma_invalid_6_half==True)
# # plevdata_ma_invalid.data.shape
# # plevdata.shape

<xarray.DataArray 'U' (time: 3, plev: 1, ncol: 777602)>
dask.array<_vertical_remap, shape=(3, 1, 777602), dtype=float32, chunksize=(3, 1, 50), chunktype=numpy.ndarray>
Coordinates:
  * plev     (plev) float32 8.5e+04
  * time     (time) object 2084-01-01 03:00:00 ... 2084-01-01 09:00:00
Dimensions without coordinates: ncol
Attributes:
    mdims:         1
    units:         m/s
    long_name:     Zonal wind
    cell_methods:  time: mean

## Fill with gridfill interpolation

This is an alternative option to filling with UBOT

In [23]:
# conda install -c conda-forge gridfill
# https://github.com/ajdawson/gridfill
# https://ocefpaf.github.io/python4oceanographers/blog/2014/10/20/gridfill/

In [24]:
# # poisson_grid_fill in nans
# import gridfill
# from gridfill import fill

In [25]:
# # may also have to crop out first few values if strange values

# # test to see if gridfill works:
# filled, converged = fill(plevdata_ma_invalid, 2, 0, 0.001, relax=0.6, itermax=4000, initzonal=False, cyclic=False, verbose=True)
# # relaxation converged w/ relax=0.6 and itermax=5000

[0] relaxation converged (3672 iterations with maximum residual 9.998e-04)


## create netCDF file

In [1]:
# if you do want to overwrite file, need to remove file first

In [ ]:
# rm /glade/scratch/tking/cgnet/test_filled.nc

In [18]:
# make a netcdf file
from netCDF4 import Dataset
import os
from datetime import datetime

# CHANGE TO WITHIN FOR STATEMENT WITH ALL FILES ONCE WORKING ON MULTIPLE!
# filename_to_create = '/glade/scratch/tking/cgnet/'+unew_files[0]
filename_to_create = '/glade/scratch/tking/cgnet/ubot_filled.nc'
input_data = merged_full
# input_data = filled
# input_data = merged_full

def create_U850_nc_file(filename_to_create, input_data):
    """Create netCDF file for U850 with a particular filename and data to input"""
    if os.path.exists(filename_to_create):
        print('warning, path exists already! not overwriting')
    else:
        ncfile = Dataset(filename_to_create, mode='w', format='NETCDF4_CLASSIC')
        lat_dim = ncfile.createDimension('time', 3)
        lon_dim = ncfile.createDimension('plev', 1)
        # lon_dim = ncfile.createDimension('plev', 30)
        time_dim = ncfile.createDimension('ncol', 777602)

        # add data to netcdf file under 'U850' or 'V850'
        plev_var = ncfile.createVariable('U850',np.float64,('time','plev','ncol')) # unlimited dimension is leftmost
        plev_var.units = 'hPa'
        plev_var.standard_name = 'zonal_wind_850hPa'

        plev_var[:,:,:] = input_data

        # other global attributes needed for netcdf files:
        data_title = "3-hr averaged/Zonal Wind/850 hPa"
        data_summary = "Create input for ClimateNet application, Poisson Fill for NaNs"
        data_creator = "Teagan King, tking@ucar.edu"
        cesm_contact = "Teagan King, tking@ucar.edu"
        data_script = "jupyter notebook at /glade/u/home/tking/cgnet/Data_Processing.ipynb"
        source_file =  ufilenames[0] # ufilename+" and "+pfilename
        conventions = "CF 1.0"
        creation_date = datetime.today().strftime('%Y-%m-%d')

        # actually add this info into file...
        ncfile.title = data_title
        ncfile.summary = data_summary
        ncfile.creator = data_creator
        ncfile.contact = cesm_contact
        ncfile.script = data_script
        # ncfile.source = source_file
        ncfile.conventions = conventions
        ncfile.creation_date = creation_date
        
        ncfile.close()

        print("{} has been written".format(filename_to_create))

create_U850_nc_file(filename_to_create, input_data)

In [31]:
# types of data that can go in nc file:
print(type(merged_full))
print(type(filled))

<class 'numpy.ma.core.MaskedArray'>
<class 'numpy.ndarray'>


## remap and view netcdf file-- commands for casper terminal

In [ ]:
# ncremap -m /glade/scratch/jet/tking/map_ne120_to_0.23x0.31_bilinear.nc -i /glade/scratch/tking/cgnet/test_filled.nc -o /glade/scratch/tking/cgnet/test_filled_regridded.nc
# ncview /glade/scratch/tking/cgnet/test_filled_regridded.nc

# ncremap -m /glade/scratch/jet/tking/map_ne120_to_0.23x0.31_bilinear.nc -i /glade/scratch/tking/cgnet/ubot_filled.nc -o /glade/scratch/tking/cgnet/ubot_filled_regridded.nc
# ncview /glade/scratch/tking/cgnet/ubot_filled_regridded.nc

In [ ]:
# !ncdump -h '/glade/scratch/tking/cgnet/b.e13.BRCP85C5CN.ne120_g16.003a.cam.h4.U850.2097010100Z-2097123121Z.nc'